In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []

In [3]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [4]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4745:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data_export = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [5]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&time=from+2015-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [6]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
china_data_import = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [7]:
china_data_export["YTD Value"] =pd.to_numeric(china_data_export["YTD Value"])

In [8]:
first_2015 = china_data_export[china_data_export["Period"].str.contains("2015")]

In [9]:
data_2015 = first_2015.groupby(["Comm Code","Commodity"])["YTD Value"].sum()

In [10]:
test= pd.DataFrame({"total" : data_2015})

In [12]:
data_2015= test.nlargest(50,"total")


In [13]:
data_2015 = data_2015.reset_index()

In [14]:
data_2015= data_2015.nlargest(50,"total")
data_2015

,Comm Code,Commodity,total
0,8800,"CIVILIAN AIRCRAFT, ENGINES, AND PARTS",16173559860
1,1201,"SOYBEANS, WHETHER OR NOT BROKEN",11707154759
2,8703,MOTOR CARS & VEHICLES FOR TRANSPO,9787269691
3,8542,"ELECTRONIC INTEGRATED CIRCUITS & MICROASSEMBL,...",3937127797
4,8517,ELECTRIC APPARATUS FOR LINE TEL,2485600493
5,1007,GRAIN SORGHUM,2078016499
6,5201,"COTTON, NOT CARDED OR COMBED",1966348354
7,8486,MACH/APPS FOR MANUFCT OF SEMIC,1937503541
8,8708,PARTS & ACCESS FOR MOTOR,1915389835
9,8471,AUTOMATIC DATA PROCESS MACHINES;,1844323017


In [15]:
data_2015 = data_2015.to_csv("data_2015.csv", index=False)